# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [5]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/cantemizyurek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/cantemizyurek/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [6]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [7]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [8]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [4]:
!mkdir data

In [5]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31493    0 31493    0     0  84712      0 --:--:-- --:--:-- --:--:-- 84886


In [6]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70519    0 70519    0     0   305k      0 --:--:-- --:--:-- --:--:--  306k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [9]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [10]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [11]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [14]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [15]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 14, relationships: 59)

We can save and load our knowledge graphs as follows.

In [16]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 14, relationships: 59)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [17]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [18]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

#### Answer:

Ragas uses different query synthesizers to generate diverse test questions from documents. Common types include Single-Hop queries, which retrieve facts from a specific context chunk, and Multi-Hop queries, which require reasoning and combining information across multiple chunks. Synthesizers can also create variations like Specific factual questions versus more Abstract or comparative ones, aiming to test the RAG system's understanding comprehensively.

Finally, we can use our `TestSetGenerator` to generate our testset!

In [19]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,what is large language models and how they is ...,[We don’t yet know how to build GPT-4 Vibes Ba...,Large Language Models is a new kind of softwar...,single_hop_specifc_query_synthesizer
1,who andy baio be?,[I’m surprised that no-one has beaten the now ...,andy baio and me did first big story on unlice...,single_hop_specifc_query_synthesizer
2,What notable developments regarding LLMs occur...,[Simon Willison’s Weblog Subscribe Stuff we fi...,2023 was the breakthrough year for Large Langu...,single_hop_specifc_query_synthesizer
3,How has Microsoft been involved in recent disc...,[Microsoft over this issue. The 69 page PDF is...,The context mentions an ongoing issue involvin...,single_hop_specifc_query_synthesizer
4,so like openai was best with gpt-4 but now not...,[The rise of inference-scaling “reasoning” mod...,"openai used to be on top with gpt-4, but now t...",single_hop_specifc_query_synthesizer
5,how much better for environment is llm now and...,[<1-hop>\n\nThe rise of inference-scaling “rea...,llm now use much less energy for each prompt b...,multi_hop_abstract_query_synthesizer
6,wut iz da relashunship betwen da enviromental ...,[<1-hop>\n\nThe rise of inference-scaling “rea...,da enviromental impact uv LLM trainin an infer...,multi_hop_abstract_query_synthesizer
7,How has the use of synthetic data influenced t...,[<1-hop>\n\nWe don’t yet know how to build GPT...,The use of synthetic data has become increasin...,multi_hop_abstract_query_synthesizer
8,How has Meta contributed to the recent advance...,[<1-hop>\n\nThe rise of inference-scaling “rea...,Meta has played a significant role in recent L...,multi_hop_specific_query_synthesizer
9,what anthopic do about inference scaling model...,[<1-hop>\n\nskeptical as to their utility base...,anthropic not release inference scaling models...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [20]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [21]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,"Which organizations, including Microsoft Resea...",[We don’t yet know how to build GPT-4 Vibes Ba...,"According to the context, organizations that h...",single_hop_specifc_query_synthesizer
1,Is AGI necessary to fully solve the gullibilit...,[I’m surprised that no-one has beaten the now ...,"According to the context, the author suspects ...",single_hop_specifc_query_synthesizer
2,Whaat is the main advancemant in Artifical Int...,[Simon Willison’s Weblog Subscribe Stuff we fi...,The main advancement in Artificial Intelligenc...,single_hop_specifc_query_synthesizer
3,As an AI Product Manager focused on leveraging...,[Microsoft over this issue. The 69 page PDF is...,The Django SQL Dashboard was used to generate ...,single_hop_specifc_query_synthesizer
4,How has open-source and community-driven devel...,[<1-hop>\n\nWe don’t yet know how to build GPT...,Open-source and community-driven development h...,multi_hop_abstract_query_synthesizer
5,How has the unpredictability of prompt enginee...,[<1-hop>\n\nI’m surprised that no-one has beat...,"The unpredictability of prompt engineering, de...",multi_hop_abstract_query_synthesizer
6,How has the unpredictability of prompt enginee...,[<1-hop>\n\nI’m surprised that no-one has beat...,The unpredictability of prompt engineering—whe...,multi_hop_abstract_query_synthesizer
7,How has open-source and community-driven devel...,[<1-hop>\n\nWe don’t yet know how to build GPT...,Open-source and community-driven development h...,multi_hop_abstract_query_synthesizer
8,How did Google’s Gemini series contribute to t...,[<1-hop>\n\nyou talk to me exclusively in Span...,"Google’s Gemini series, particularly Gemini 1....",multi_hop_specific_query_synthesizer
9,How did Google's Gemini 1.5 Pro contribute to ...,[<1-hop>\n\nThe rise of inference-scaling “rea...,"Google's Gemini 1.5 Pro, released in February,...",multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [22]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [23]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [24]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [26]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [27]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

/Users/cantemizyurek/Documents/code/ai/ai-maker-space/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/qdrant_client/http/models/models.py:758: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is empty, alternative syntax for `is_empty: \&quot;field_name\&quot;`",
/Users/cantemizyurek/Documents/code/ai/ai-maker-space/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/qdrant_client/http/models/models.py:762: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is null, alternative syntax for `is_null: \&quot;field_name\&quot;`",


In [28]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [29]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [30]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1")

Finally, we can set-up our RAG LCEL chain!

In [31]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [32]:
rag_chain.invoke({"question" : "What are Agents?"})

'Based on the provided context, "agents" is an infuriatingly vague term that lacks a single, clear, and widely understood meaning. The term "agents" is used to describe concepts such as:\n\n- AI systems that can go away and act on your behalf (e.g., the travel agent model).\n- LLMs (large language models) that have been given access to tools which they can run in a loop as part of solving a problem.\n- Systems that act with a degree of "autonomy," though the definition of autonomy is also unclear.\n\nOverall, the context emphasizes that the term "agents" conveys little information without further clarification, and people using it may be referring to different things. There is no consensus on a precise definition.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [33]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [34]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `dope_or_nope_evaluator`:

#### Answer:

qa_evaluator: This is a standard LangSmith evaluator. It evaluates the quality and correctness of the generated answer (prediction) by comparing it against a provided ground truth or reference answer.

labeled_helpfulness_evaluator: This is a custom criteria-based evaluator. It uses a specific criterion to judge if the generated answer is helpful to the user, specifically taking into account the correct reference answer.

dope_or_nope_evaluator: This is another custom criteria-based evaluator. It evaluates the generated answer (prediction) based only on the subjective criterion of whether it is "dope or not?"

## LangSmith Evaluation

In [35]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'healthy-value-11' at:
https://smith.langchain.com/o/51426868-c591-4842-82cc-98382a04e3b2/datasets/a8adca5d-482a-44ee-9e92-b9e6bea7a6c7/compare?selectedSessions=898c28bc-d9f7-4e09-bd2f-4b2256208bdb




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,"how google do with llms now, like what new stu...",Based on the provided context:\n\n**How is Goo...,None,"google now got gemini 1.5 pro, it come out in ...",1,1,0,15.825690,f0d9f417-acf7-46bf-84e9-450425032005,76a5dc87-2019-41e1-8ab4-249383cae412
1,why lLMs so cheap now and what problem with lL...,LLMs are so cheap now because of two main fact...,None,llMs so cheap now because more competition and...,1,1,1,4.196383,ef07eecc-a5f8-4bf7-97c8-b9713a4f13c0,4099ca4f-b690-4834-92e5-204d3bfbefc8
2,How did Google's Gemini 1.5 Pro contribute to ...,Google's Gemini 1.5 Pro contributed to the tre...,None,"Google's Gemini 1.5 Pro, released in February,...",1,1,0,4.701025,4426e692-1ca7-4e01-8e96-0503a877f6f9,5bb07b73-3f71-4b83-90e4-7f48faf6d99d
3,How did Google’s Gemini series contribute to t...,Google’s Gemini series contributed to the tren...,None,"Google’s Gemini series, particularly Gemini 1....",1,1,0,4.707349,121ab3ed-25b5-4f6f-9f16-5942f36d0c5b,2984704d-3775-470e-93bf-90cb28a518f2
4,How has open-source and community-driven devel...,Open-source and community-driven development h...,None,Open-source and community-driven development h...,1,1,1,5.934337,ecf5091e-4d1f-4472-b606-f60f361b3ee9,25178816-13ee-418b-b68e-2e963584e8a8
5,How has the unpredictability of prompt enginee...,Based on the provided context:\n\nThe unpredic...,None,The unpredictability of prompt engineering—whe...,1,0,0,6.991659,1784f707-9660-41ff-933e-cf39009191b9,e9a28a0f-d2c6-42be-9351-a7250a4ed7f5
6,How has the unpredictability of prompt enginee...,"Based only on the provided context, here is wh...",None,"The unpredictability of prompt engineering, de...",1,1,0,6.467208,3a47878e-0f57-42b7-abdb-cb01a5ad02c0,6d4e9c17-bf58-44c0-b515-4b8768b72375
7,How has open-source and community-driven devel...,Open-source and community-driven development h...,None,Open-source and community-driven development h...,1,1,0,7.197557,6de6528f-e285-4df5-af01-667e0cb904f8,17c62b80-eec7-4dc7-9ce2-e48a6c2ae89b
8,As an AI Product Manager focused on leveraging...,"Based on the provided context, the Django SQL ...",None,The Django SQL Dashboard was used to generate ...,1,1,0,3.003762,d9d71c4d-f108-4033-9bd3-15a386601a4d,3cecdbb3-ce19-46ba-8e71-6c643476eda6
9,Whaat is the main advancemant in Artifical Int...,The main advancement in Artificial Intelligenc...,None,The main advancement in Artificial Intelligenc...,1,1,0,1.278243,c918af1a-b26e-4a5d-86e7-493817396a2e,151e53da-3bb0-4d84-8482-c86a83dd88fa


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [36]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [37]:
rag_documents = docs

In [38]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

#### Answer:

Chunk size directly impacts RAG performance by determining the amount of context retrieved. Smaller chunks offer higher precision for specific facts but may lack broader context, while larger chunks capture more context but risk retrieving irrelevant information (noise) that can dilute accuracy and hinder the language model's ability to generate a concise answer. Finding the optimal chunk size balances context and precision for the specific task and data.

In [39]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

#### Answer:

The embedding model is critical as it translates text into numerical vectors representing semantic meaning. A different model may capture the nuances and relationships within your specific documents more (or less) effectively, directly impacting the quality of retrieval – good embeddings lead to relevant context being found, while poor embeddings result in irrelevant context, significantly affecting the final answer quality and overall RAG performance.

In [40]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [41]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [42]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [43]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

'Alright, here’s the lowdown: the term “agents” in the AI world is super fuzzy—like, it means different things to different people and doesn’t have a clear, widely agreed definition. For some folks, “agents” are supposed to be AI systems that act on your behalf, like a travel agent that goes out and gets stuff done for you. Other people mean LLMs (large language models) that can use tools or run in loops to solve problems. Sometimes, people throw around words like “autonomy” but still don’t get specific about what they really mean.\n\nBottom line: “agents” are basically AI systems expected to take actions or make decisions for you, but the exact meaning totally depends on who’s talking. And as of now, despite all the hype, they’re still more “coming soon” than reality, mainly because of challenges like “gullibility”—AI’s tendency to believe anything—which makes true agent behavior tough to pull off.\n\nSo, until everyone agrees what the heck an “agent” actually is and can make them wor

Finally, we can evaluate the new chain on the same test set!

In [44]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'only-rain-80' at:
https://smith.langchain.com/o/51426868-c591-4842-82cc-98382a04e3b2/datasets/a8adca5d-482a-44ee-9e92-b9e6bea7a6c7/compare?selectedSessions=ed379608-14dd-4ead-9286-4eeef937c027




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,"how google do with llms now, like what new stu...","Alright, let's get into it! Google’s really up...",None,"google now got gemini 1.5 pro, it come out in ...",1,1,1,12.584633,f0d9f417-acf7-46bf-84e9-450425032005,b209aec0-25c4-4543-89d9-639a97f47041
1,why lLMs so cheap now and what problem with lL...,"Alright, let’s break it down in style:\n\nWhy ...",None,llMs so cheap now because more competition and...,1,0,1,5.980305,ef07eecc-a5f8-4bf7-97c8-b9713a4f13c0,0667eeb6-2f95-4207-8aee-bd0f80619d48
2,How did Google's Gemini 1.5 Pro contribute to ...,Yo! Gemini 1.5 Pro rolled into the LLM scene l...,None,"Google's Gemini 1.5 Pro, released in February,...",1,1,1,6.870791,4426e692-1ca7-4e01-8e96-0503a877f6f9,22bf4e9d-d6d8-4267-9149-233477b20743
3,How did Google’s Gemini series contribute to t...,"Yo, check it—Google’s Gemini series pulled up ...",None,"Google’s Gemini series, particularly Gemini 1....",1,1,1,9.085816,121ab3ed-25b5-4f6f-9f16-5942f36d0c5b,319a1654-dd64-4264-a646-71c6def8e071
4,How has open-source and community-driven devel...,"Yo, let me break it down for you! Open-source ...",None,Open-source and community-driven development h...,1,1,1,9.933330,ecf5091e-4d1f-4472-b606-f60f361b3ee9,283ead40-1122-4a18-a62f-faba42324f5e
5,How has the unpredictability of prompt enginee...,"Yo, let’s break it down!\n\nThe unpredictabili...",None,The unpredictability of prompt engineering—whe...,1,1,1,8.178475,1784f707-9660-41ff-933e-cf39009191b9,536f00a0-6c8e-41be-8a66-8728b5949be5
6,How has the unpredictability of prompt enginee...,"Yo, check it—2024’s been wild for prompt-drive...",None,"The unpredictability of prompt engineering, de...",1,1,1,8.192782,3a47878e-0f57-42b7-abdb-cb01a5ad02c0,85ca7da9-7414-4a87-9b56-64c9031eb8f3
7,How has open-source and community-driven devel...,Yo! The open-source and community-driven movem...,None,Open-source and community-driven development h...,1,1,1,7.059512,6de6528f-e285-4df5-af01-667e0cb904f8,d5189156-49d6-4c53-be86-f59680ff6784
8,As an AI Product Manager focused on leveraging...,"Yo, check this out—Django SQL Dashboard was th...",None,The Django SQL Dashboard was used to generate ...,1,0,1,4.813225,d9d71c4d-f108-4033-9bd3-15a386601a4d,d81c659e-0166-4361-b462-83facf006ddc
9,Whaat is the main advancemant in Artifical Int...,"Yo! In 2023, the main advancement in Artificia...",None,The main advancement in Artificial Intelligenc...,1,1,1,2.390381,c918af1a-b26e-4a5d-86e7-493817396a2e,4c7c9bf7-c840-47c6-88cc-4bb94aab025b


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.